In [ ]:
!pip install nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


change "CollSgE" or whatever the dialect is to the one you want

In [ ]:
import pandas as pd
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import os

# Function to calculate BLEU score for a pair of sentences
def calculate_bleu(reference, hypothesis):
    if not reference or not hypothesis:  # Handle empty strings
        return 0.0
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing_function)

# Calculate BLEU for Logic Bench MCQ
def process_logic_bench_mcq(file_path, output_directory, output_file_name, filtered_file_name):
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract relevant data
    extracted_data = []
    for i, sample in enumerate(data.get("propositional_logic", {}).get("samples", [])):
        context = sample.get("context", "")
        translated_context = sample.get("IndE (context)", "")

        if not context or not translated_context:
            print(f"Warning: Missing data in sample {i + 1} - Context: {context}, IndE Context: {translated_context}")
            continue

        extracted_data.append({"reference": context, "hypothesis": translated_context})

    # Calculate BLEU scores
    df = pd.DataFrame(extracted_data)
    df["BLEU Score"] = df.apply(lambda row: calculate_bleu(row["reference"], row["hypothesis"]), axis=1)

    # Save results
    os.makedirs(output_directory, exist_ok=True)
    df.to_csv(os.path.join(output_directory, output_file_name), index=False)
    df[df["BLEU Score"] < 0.7].to_csv(os.path.join(output_directory, filtered_file_name), index=False)

    print(f"Processed Logic Bench MCQ: {file_path}")

# Calculate BLEU for Logic Bench YN
def process_logic_bench_yn(file_path, output_directory, output_file_name, filtered_file_name):
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract relevant data
    extracted_data = []
    for sample in data.get("propositional_logic", {}).get("samples", []):
        context = sample.get("context", "")
        translated_context = sample.get("IndE (context)", "")

        if not context or not translated_context:
            print(f"Warning: Missing data in sample - Context: {context}, IndE Context: {translated_context}")
            continue

        extracted_data.append({"reference": context, "hypothesis": translated_context})

    # Calculate BLEU scores
    df = pd.DataFrame(extracted_data)
    df["BLEU Score"] = df.apply(lambda row: calculate_bleu(row["reference"], row["hypothesis"]), axis=1)

    # Save results
    os.makedirs(output_directory, exist_ok=True)
    df.to_csv(os.path.join(output_directory, output_file_name), index=False)
    df[df["BLEU Score"] < 0.7].to_csv(os.path.join(output_directory, filtered_file_name), index=False)

    print(f"Processed Logic Bench YN: {file_path}")

# File paths and configurations for both tasks
datasets = {
    "Logic Bench MCQ": {
        "file_path": "/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/GPT 4o/IndE/Logic Bench MCQ(480)_IndE.json",
        "output_directory": "/content/drive/MyDrive/!!Multi-AAVENUE/BLEU Score Datasets/IndE/Logic Bench MCQ",
        "output_file_name": "logic_bench_mcq_bleu_scores.csv",
        "filtered_file_name": "logic_bench_mcq_filtered_bleu_scores.csv",
    },
    "Logic Bench YN": {
        "file_path": "/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/GPT 4o/IndE/Logic Bench YN(500)_IndE.json",
        "output_directory": "/content/drive/MyDrive/!!Multi-AAVENUE/BLEU Score Datasets/IndE/Logic Bench YN",
        "output_file_name": "logic_bench_yn_bleu_scores.csv",
        "filtered_file_name": "logic_bench_yn_filtered_bleu_scores.csv",
    },
}

# Process both datasets
try:
    mcq_info = datasets["Logic Bench MCQ"]
    process_logic_bench_mcq(
        file_path=mcq_info["file_path"],
        output_directory=mcq_info["output_directory"],
        output_file_name=mcq_info["output_file_name"],
        filtered_file_name=mcq_info["filtered_file_name"]
    )
except ValueError as e:
    print(f"Error processing Logic Bench MCQ: {e}")

try:
    yn_info = datasets["Logic Bench YN"]
    process_logic_bench_yn(
        file_path=yn_info["file_path"],
        output_directory=yn_info["output_directory"],
        output_file_name=yn_info["output_file_name"],
        filtered_file_name=yn_info["filtered_file_name"]
    )
except ValueError as e:
    print(f"Error processing Logic Bench YN: {e}")

print("Processing completed.")


Processed Logic Bench MCQ: /content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/GPT 4o/IndE/Logic Bench MCQ(480)_IndE.json
Processed Logic Bench YN: /content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/GPT 4o/IndE/Logic Bench YN(500)_IndE.json
Processing completed.
